In [1]:
from __future__ import absolute_import, division, print_function

import argparse
import collections
import json
import logging
import enum
import os
import random
import sys
sys.path.append("./")

from io import open

import numpy as np
import torch
from tqdm import tqdm

from pytorch_pretrained_bert.tokenization import BertTokenizer
from glob import glob
import tensorflow as tf
import gzip
import multiprocessing

if sys.version_info[0] == 2:
    import cPickle as pickle
else:
    import pickle

import six

from spacy.lang.en import English
from modules.graph_encoder import NodePosition, Graph, EdgeType, get_edge_position

In [2]:
_ = 3
print(os.path.join("checkpts","a.bin"+"."+str(_)))

checkpts/a.bin.3


In [3]:
import gzip
import tensorflow as tf
import six
import json
from src_nq.create_examples import NqExample
input_path = "./natural_questions/v1.0/sample/nq-train-sample.jsonl.gz"
fi = gzip.GzipFile(fileobj=tf.io.gfile.GFile(input_path, "rb"))

In [5]:
examples = []
for line in fi:
    if not isinstance(line, six.text_type):
        line = line.decode("utf-8")
    json_example = json.loads(line)
    question_tokens = json_example["question_tokens"]
    if question_tokens == None:
        continue
    doc_tokens = json_example["document_tokens"]
    annotation = json_example["annotations"]
    example_id = json_example["example_id"]
    la_candidates = json_example["long_answer_candidates"]

    examples.append(
        NqExample(example_id=example_id,
                    question_tokens=question_tokens,
                    doc_tokens=doc_tokens,
                    la_candidates=la_candidates,
                    annotation=annotation))


In [6]:
examples[0].question_tokens

['in',
 'greek',
 'mythology',
 'who',
 'was',
 'the',
 'goddess',
 'of',
 'spring',
 'growth']

In [7]:
examples[0].doc_tokens[:20]

[{'end_byte': 102,
  'html_token': False,
  'start_byte': 92,
  'token': 'Persephone'},
 {'end_byte': 104, 'html_token': False, 'start_byte': 103, 'token': '-'},
 {'end_byte': 114,
  'html_token': False,
  'start_byte': 105,
  'token': 'Wikipedia'},
 {'end_byte': 58058, 'html_token': True, 'start_byte': 58005, 'token': '<H1>'},
 {'end_byte': 58068,
  'html_token': False,
  'start_byte': 58058,
  'token': 'Persephone'},
 {'end_byte': 58073,
  'html_token': True,
  'start_byte': 58068,
  'token': '</H1>'},
 {'end_byte': 58565,
  'html_token': False,
  'start_byte': 58561,
  'token': 'This'},
 {'end_byte': 58573,
  'html_token': False,
  'start_byte': 58566,
  'token': 'article'},
 {'end_byte': 58576, 'html_token': False, 'start_byte': 58574, 'token': 'is'},
 {'end_byte': 58582,
  'html_token': False,
  'start_byte': 58577,
  'token': 'about'},
 {'end_byte': 58586, 'html_token': False, 'start_byte': 58583, 'token': 'the'},
 {'end_byte': 58592,
  'html_token': False,
  'start_byte': 58587,

In [8]:
examples[0].annotation

[{'annotation_id': 7719528322202775345,
  'long_answer': {'candidate_index': 58,
   'end_byte': 84070,
   'end_token': 965,
   'start_byte': 81195,
   'start_token': 809},
  'short_answers': [{'end_byte': 82556,
    'end_token': 838,
    'start_byte': 81281,
    'start_token': 814}],
  'yes_no_answer': 'NONE'}]

In [9]:
examples[0].la_candidates[:10]

[{'end_byte': 62898,
  'end_token': 217,
  'start_byte': 58761,
  'start_token': 24,
  'top_level': True},
 {'end_byte': 58933,
  'end_token': 30,
  'start_byte': 58810,
  'start_token': 25,
  'top_level': False},
 {'end_byte': 59089,
  'end_token': 44,
  'start_byte': 58933,
  'start_token': 30,
  'top_level': False},
 {'end_byte': 59894,
  'end_token': 60,
  'start_byte': 59089,
  'start_token': 44,
  'top_level': False},
 {'end_byte': 60156,
  'end_token': 74,
  'start_byte': 59894,
  'start_token': 60,
  'top_level': False},
 {'end_byte': 60254,
  'end_token': 92,
  'start_byte': 60156,
  'start_token': 74,
  'top_level': False},
 {'end_byte': 60446,
  'end_token': 104,
  'start_byte': 60355,
  'start_token': 96,
  'top_level': False},
 {'end_byte': 60618,
  'end_token': 114,
  'start_byte': 60446,
  'start_token': 104,
  'top_level': False},
 {'end_byte': 60763,
  'end_token': 124,
  'start_byte': 60618,
  'start_token': 114,
  'top_level': False},
 {'end_byte': 62753,
  'end_toke

In [10]:
# part = examples[:10]
part = examples[:1]
part

In [11]:
from pytorch_pretrained_bert.tokenization import BertTokenizer
tokenizer = BertTokenizer(vocab_file="./bert-base-uncased-vocab.txt", do_lower_case=True)
is_training = True


In [12]:
do_lower_case=True
max_seq_length=512
doc_stride=128
max_query_length=64
include_unknowns=0.03
max_position=50
num_threads=16
seed=42
start_num=-1
end_num=-1

In [13]:
from src_nq.create_examples import *

In [14]:
def _check_is_max_context(doc_spans, cur_span_index, position):
    """Check if this is the 'max context' doc span for the token."""

    # Because of the sliding window approach taken to scoring documents, a single
    # token can appear in multiple documents. E.g.
    #  Doc: the man went to the store and bought a gallon of milk
    #  Span A: the man went to the
    #  Span B: to the store and bought
    #  Span C: and bought a gallon of
    #  ...
    #
    # Now the word 'bought' will have two scores from spans B and C. We only
    # want to consider the score with "maximum context", which we define as
    # the *minimum* of its left and right context (the *sum* of left and
    # right context will always be the same, of course).
    #
    # In the example the maximum context for 'bought' would be span C since
    # it has 1 left context and 3 right context, while span B has 4 left context
    # and 0 right context.
    best_score = None
    best_span_index = None
    for (span_index, doc_span) in enumerate(doc_spans):
        end = doc_span.start + doc_span.length - 1
        if position < doc_span.start:
            continue
        if position > end:
            continue
        num_left_context = position - doc_span.start
        num_right_context = end - position
        score = min(num_left_context, num_right_context) + 0.01 * doc_span.length
        if best_score is None or score > best_score:
            best_score = score
            best_span_index = span_index

    return cur_span_index == best_span_index

In [25]:
def test_convert(args, examples, tokenizer, is_training, cached_path):

    unique_id = 1000000000

    features = []
    for (example_index, example) in enumerate(examples):
        query_tokens = []
        for token in example.question_tokens:
            sub_tokens = tokenizer.tokenize(token)
            for sub_token in sub_tokens:
                query_tokens.append(sub_token)

        if len(query_tokens) > max_query_length:
            query_tokens = query_tokens[-max_query_length:]

        counts = {"Table": 0, "Paragraph": 0, "List": 0, "Other": 0}

        sorted_la_candidates = sorted(enumerate(example.la_candidates), key=lambda x: x[1]["start_token"])

        tok_is_sentence_end = []
        tok_is_paragraph_end = []
        tok_candidate_idx = []
        tok_to_orig_index = []
        orig_to_tok_index = [-1] * len(example.doc_tokens)
        orig_to_tok_end_index = [-1] * len(example.doc_tokens)
        all_doc_tokens = []

        for candidate_idx, candidate in sorted_la_candidates:
            if not candidate["top_level"]:
                continue
            start_index = candidate["start_token"]
            end_index = candidate["end_token"]

            context_tokens = []
            context_orig_idx = []

            for tok_idx in range(start_index, end_index):
                token_item = example.doc_tokens[tok_idx]
                if not token_item["html_token"]:
                    context_tokens.append(token_item["token"].replace(" ", ""))
                    if context_tokens[-1] == "":
                        context_tokens = context_tokens[:-1]
                        continue
                    context_orig_idx.append(tok_idx)

            if len(context_tokens) == 0:
                continue

            all_doc_tokens.append(
                "[unused{}]".format(
                    get_candidate_type(example.doc_tokens[start_index]["token"], counts, max_position)))
            tok_to_orig_index.append(-1)
            tok_is_sentence_end.append(False)
            tok_is_paragraph_end.append(False)
            tok_candidate_idx.append(candidate_idx)

            paragraph_len = 0
            context = " ".join(context_tokens)
            context_sentences = nlp(context).sents
            context_idx = 0
            orig_to_tok_index[context_orig_idx[context_idx]] = len(all_doc_tokens)
            for sentence in context_sentences:
                sentence_len = 0
                for token in sentence.string.strip().split():
                    if len(context_tokens[context_idx]) == 0:
                        orig_to_tok_end_index[context_orig_idx[context_idx]] = len(all_doc_tokens)
                        context_idx += 1
                        orig_to_tok_index[context_orig_idx[context_idx]] = len(all_doc_tokens)
                    assert context_tokens[context_idx][:len(token)] == token
                    context_tokens[context_idx] = context_tokens[context_idx][len(token):]

                    sub_tokens = tokenizer.tokenize(token)

                    all_doc_tokens += sub_tokens
                    tok_to_orig_index += [context_orig_idx[context_idx]] * len(sub_tokens)
                    sentence_len += len(sub_tokens)
                    paragraph_len += len(sub_tokens)
                if sentence_len > 0:
                    tok_is_sentence_end += [False] * (sentence_len - 1) + [True]

            orig_to_tok_end_index[context_orig_idx[context_idx]] = len(all_doc_tokens)
            assert context_idx + 1 == len(context_tokens)
            assert context_tokens[context_idx] == ""

            tok_is_paragraph_end += [False] * (paragraph_len - 1) + [True]
            tok_candidate_idx += [candidate_idx] * paragraph_len

        assert len(tok_is_sentence_end) == len(tok_is_paragraph_end)

        la_tok_start_position = None
        la_tok_end_position = None

        sa_tok_start_position = None
        sa_tok_end_position = None

        example_answer_type = None

        if is_training:
            annotation = example.annotation[0]
            if annotation["long_answer"]["start_token"] != -1:
                la_start_token = annotation["long_answer"]["start_token"]
                la_end_token = annotation["long_answer"]["end_token"] - 1
                while orig_to_tok_index[la_start_token] == -1:
                    la_start_token += 1
                while orig_to_tok_end_index[la_end_token] == -1:
                    la_end_token -= 1

                assert la_start_token <= la_end_token

                la_tok_start_position = orig_to_tok_index[la_start_token]
                la_tok_end_position = orig_to_tok_end_index[la_end_token] - 1

                if annotation["yes_no_answer"].lower() == "none":
                    example_answer_type = AnswerType.LONG
                elif annotation["yes_no_answer"].lower() == "yes":
                    example_answer_type = AnswerType.YES
                elif annotation["yes_no_answer"].lower() == "no":
                    example_answer_type = AnswerType.NO
                else:
                    assert False

                if "short_answers" in annotation and len(annotation["short_answers"]) > 0:
                    sa_tok_start_position = len(all_doc_tokens) + 1
                    sa_tok_end_position = -1
                    example_answer_type = AnswerType.SHORT
                    for short_answer in annotation["short_answers"]:
                        sa_start_token = short_answer["start_token"]
                        sa_end_token = short_answer["end_token"] - 1

                        sa_tok_start_position = min(sa_tok_start_position, orig_to_tok_index[sa_start_token])
                        sa_tok_end_position = max(sa_tok_end_position, orig_to_tok_end_index[sa_end_token] - 1)
                        assert sa_tok_start_position >= 0 and sa_tok_end_position >= 0
                    assert la_tok_start_position <= sa_tok_start_position and la_tok_end_position >= sa_tok_end_position
                    la_tok_start_position = sa_tok_start_position
                    la_tok_end_position = sa_tok_end_position
                else:
                    sa_tok_start_position = -1
                    sa_tok_end_position = -1
            else:
                example_answer_type = AnswerType.UNKNOWN
                la_tok_start_position = -1
                la_tok_end_position = -1
                sa_tok_start_position = -1
                sa_tok_end_position = -1

        # The -3 accounts for [CLS], [SEP] and [SEP]
        max_tokens_for_doc = max_seq_length - len(query_tokens) - 3

        # We can have documents that are longer than the maximum sequence length.
        # To deal with this we do a sliding window approach, where we take chunks
        # of the up to our max length with a stride of `doc_stride`.
        _DocSpan = collections.namedtuple(  # pylint: disable=invalid-name
            "DocSpan", ["start", "length"])
        doc_spans = []
        start_offset = 0
        logger.info("ALL DOC_TOKENS %s" %" ".join([str(x) for x in all_doc_tokens][:50]))
        while start_offset < len(all_doc_tokens):
            length = len(all_doc_tokens) - start_offset
            if length > max_tokens_for_doc:
                length = max_tokens_for_doc
            doc_spans.append(_DocSpan(start=start_offset, length=length))
            if start_offset + length == len(all_doc_tokens):
                break
            start_offset += min(length, doc_stride)

        for (doc_span_index, doc_span) in enumerate(doc_spans):
            tokens = []
            token_to_orig_map = {}
            token_is_max_context = {}
            segment_ids = []
            tokens.append("[CLS]")
            segment_ids.append(0)
            for token in query_tokens:
                tokens.append(token)
                segment_ids.append(0)
            tokens.append("[SEP]")
            segment_ids.append(0)

            is_sentence_end = [True] + [False] * (len(tokens) - 3) + [True] + [False]
            is_paragraph_end = [True] + [False] * (len(tokens) - 3) + [True] + [False]
            orig_tok_idx = [-1] * len(tokens)
            candidate_idx = [-1] * len(tokens)

            for i in range(doc_span.length):
                split_token_index = doc_span.start + i
                token_to_orig_map[len(tokens)] = tok_to_orig_index[split_token_index]

                is_max_context = _check_is_max_context(doc_spans, doc_span_index,
                                                       split_token_index)
                token_is_max_context[len(tokens)] = is_max_context
                tokens.append(all_doc_tokens[split_token_index])
                segment_ids.append(1)

                is_sentence_end.append(tok_is_sentence_end[split_token_index])
                is_paragraph_end.append(tok_is_paragraph_end[split_token_index])
                orig_tok_idx.append(tok_to_orig_index[split_token_index])
                candidate_idx.append(tok_candidate_idx[split_token_index])

            is_sentence_end[-1] = False
            is_paragraph_end[-1] = False

            tokens.append("[SEP]")
            segment_ids.append(1)

            is_sentence_end.append(True)
            is_paragraph_end.append(True)
            orig_tok_idx.append(-1)
            candidate_idx.append(-1)

            assert len(orig_tok_idx) == len(tokens)
            assert len(is_sentence_end) == len(tokens)
            assert len(is_paragraph_end) == len(tokens)
            assert len(candidate_idx) == len(tokens)

            input_ids = tokenizer.convert_tokens_to_ids(tokens)

            # The mask has 1 for real tokens and 0 for padding tokens. Only real
            # tokens are attended to.
            input_mask = [1] * len(input_ids)

            # Zero-pad up to the sequence length.
            while len(input_ids) < max_seq_length:
                input_ids.append(0)
                input_mask.append(0)
                segment_ids.append(0)

            assert len(input_ids) == max_seq_length
            assert len(input_mask) == max_seq_length
            assert len(segment_ids) == max_seq_length

            la_start_position = None
            la_end_position = None
            sa_start_position = None
            sa_end_position = None
            answer_type = None
            if is_training:
                # For training, if our document chunk does not contain an annotation
                # we throw it out, since there is nothing to predict.
                doc_start = doc_span.start
                doc_end = doc_span.start + doc_span.length - 1
                out_of_span = False
                if not (la_tok_start_position >= doc_start and
                        la_tok_end_position <= doc_end):
                    out_of_span = True

                if out_of_span:
                    if random.random() > include_unknowns:
                        continue
                    answer_type = AnswerType.UNKNOWN
                    la_start_position = 0
                    la_end_position = 0
                    sa_start_position = 0
                    sa_end_position = 0
                else:
                    answer_type = example_answer_type
                    doc_offset = len(query_tokens) + 2
                    la_start_position = la_tok_start_position - doc_start + doc_offset
                    la_end_position = la_tok_end_position - doc_start + doc_offset

                    if not (sa_tok_start_position >= doc_start and sa_tok_end_position <= doc_end):
                        sa_start_position = -1
                        sa_end_position = -1
                    else:
                        sa_start_position = sa_tok_start_position - doc_start + doc_offset
                        sa_end_position = sa_tok_end_position - doc_start + doc_offset

                assert la_start_position <= la_end_position
                assert sa_start_position <= sa_end_position

            if example_index >= 0:
                logger.info("*** Example ***")
                logger.info("unique_id: %s" % (unique_id))
                logger.info("example_index: %s" % (example_index))
                logger.info("doc_span_index: %s" % (doc_span_index))
                print("token list:",tokens[:20])
                # print("token list size", len(tokens))
                logger.info("tokens: %s" % " ".join(tokens)[:20])
                logger.info("token_to_orig_map: %s" % " ".join([
                    "%d:%d" % (x, y) for (x, y) in token_to_orig_map.items()][:20]))
                logger.info("token_is_max_context: %s" % " ".join([
                    "%d:%s" % (x, y) for (x, y) in token_is_max_context.items()
                ][:20]))
                logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids][:20]))
                logger.info(
                    "input_mask: %s" % " ".join([str(x) for x in input_mask][:20]))
                logger.info(
                    "segment_ids: %s" % " ".join([str(x) for x in segment_ids][:20]))
                if is_training and answer_type != AnswerType.UNKNOWN:
                    answer_text = " ".join(tokens[la_start_position:(la_end_position + 1)][:20])
                    logger.info("la_start_position: %d" % (la_start_position))
                    logger.info("la_end_position: %d" % (la_end_position))
                    logger.info(
                        "answer: %s" % (answer_text))
            doc_tree = get_doc_tree(is_sentence_end, is_paragraph_end, orig_tok_idx, candidate_idx)

            graph, start_positions, end_positions = build_graph(doc_tree, len(is_sentence_end),
                                                                la_start_position, la_end_position,
                                                                sa_start_position, sa_end_position)

            features.append(
                InputFeatures(
                    unique_id=unique_id,
                    example_id=example.example_id,
                    doc_span_index=doc_span_index,
                    tokens=None,
                    token_to_orig_map=token_to_orig_map,
                    token_is_max_context=token_is_max_context,
                    input_ids=input_ids,
                    input_mask=input_mask,
                    segment_ids=segment_ids,
                    graph=graph,
                    answer_type=answer_type,
                    start_positions=start_positions,
                    end_positions=end_positions))
            unique_id += 1

    return features



In [26]:
features = test_convert(None,part, tokenizer, is_training,"")

12/30/2020 15:32:20 - INFO - src_nq.create_examples -   ALL DOC_TOKENS [unused0] per ##se ##phone goddess of the underworld , spring ##time , flowers and vegetation statue of per ##se ##phone with a sis ##trum . her ##ak ##lion archaeological museum , crete ab ##ode the underworld , sicily , mount olympus symbol po ##me ##gra ##nate , seeds of grain
12/30/2020 15:32:20 - INFO - src_nq.create_examples -   *** Example ***
12/30/2020 15:32:20 - INFO - src_nq.create_examples -   unique_id: 1000000000
12/30/2020 15:32:20 - INFO - src_nq.create_examples -   example_index: 0
12/30/2020 15:32:20 - INFO - src_nq.create_examples -   doc_span_index: 2
12/30/2020 15:32:20 - INFO - src_nq.create_examples -   tokens: [CLS] in greek mytho
12/30/2020 15:32:20 - INFO - src_nq.create_examples -   token_to_orig_map: 12:303 13:303 14:306 15:306 16:306 17:309 18:309 19:309 20:309 21:309 22:316 23:316 24:317 25:318 26:319 27:323 28:323 29:330 30:330 31:330
12/30/2020 15:32:20 - INFO - src_nq.create_examples

In [28]:
examples = part
is_training = True
cached_path = ""
unique_id = 1000000000

features = []
for (example_index, example) in enumerate(examples):
    query_tokens = []
    for token in example.question_tokens:
        sub_tokens = tokenizer.tokenize(token)
        for sub_token in sub_tokens:
            query_tokens.append(sub_token)

    if len(query_tokens) > max_query_length:
        query_tokens = query_tokens[-max_query_length:]

    counts = {"Table": 0, "Paragraph": 0, "List": 0, "Other": 0}

    sorted_la_candidates = sorted(enumerate(example.la_candidates), key=lambda x: x[1]["start_token"])

    tok_is_sentence_end = []
    tok_is_paragraph_end = []
    tok_candidate_idx = []
    tok_to_orig_index = []
    orig_to_tok_index = [-1] * len(example.doc_tokens)
    orig_to_tok_end_index = [-1] * len(example.doc_tokens)
    all_doc_tokens = []

    for candidate_idx, candidate in sorted_la_candidates:
        if not candidate["top_level"]:
            continue
        start_index = candidate["start_token"]
        end_index = candidate["end_token"]

        context_tokens = []
        context_orig_idx = []

        for tok_idx in range(start_index, end_index):
            token_item = example.doc_tokens[tok_idx]
            if not token_item["html_token"]:
                context_tokens.append(token_item["token"].replace(" ", ""))
                if context_tokens[-1] == "":
                    context_tokens = context_tokens[:-1]
                    continue
                context_orig_idx.append(tok_idx)

        if len(context_tokens) == 0:
            continue

        all_doc_tokens.append(
            "[unused{}]".format(
                get_candidate_type(example.doc_tokens[start_index]["token"], counts, max_position)))
        tok_to_orig_index.append(-1)
        tok_is_sentence_end.append(False)
        tok_is_paragraph_end.append(False)
        tok_candidate_idx.append(candidate_idx)

        paragraph_len = 0
        context = " ".join(context_tokens)
        context_sentences = nlp(context).sents
        context_idx = 0
        orig_to_tok_index[context_orig_idx[context_idx]] = len(all_doc_tokens)
        for sentence in context_sentences:
            sentence_len = 0
            for token in sentence.string.strip().split():
                if len(context_tokens[context_idx]) == 0:
                    orig_to_tok_end_index[context_orig_idx[context_idx]] = len(all_doc_tokens)
                    context_idx += 1
                    orig_to_tok_index[context_orig_idx[context_idx]] = len(all_doc_tokens)
                assert context_tokens[context_idx][:len(token)] == token
                context_tokens[context_idx] = context_tokens[context_idx][len(token):]

                sub_tokens = tokenizer.tokenize(token)

                all_doc_tokens += sub_tokens
                tok_to_orig_index += [context_orig_idx[context_idx]] * len(sub_tokens)
                sentence_len += len(sub_tokens)
                paragraph_len += len(sub_tokens)
            if sentence_len > 0:
                tok_is_sentence_end += [False] * (sentence_len - 1) + [True]

        orig_to_tok_end_index[context_orig_idx[context_idx]] = len(all_doc_tokens)
        assert context_idx + 1 == len(context_tokens)
        assert context_tokens[context_idx] == ""

        tok_is_paragraph_end += [False] * (paragraph_len - 1) + [True]
        tok_candidate_idx += [candidate_idx] * paragraph_len

    assert len(tok_is_sentence_end) == len(tok_is_paragraph_end)

    la_tok_start_position = None
    la_tok_end_position = None

    sa_tok_start_position = None
    sa_tok_end_position = None

    example_answer_type = None

    if is_training:
        annotation = example.annotation[0]
        if annotation["long_answer"]["start_token"] != -1:
            la_start_token = annotation["long_answer"]["start_token"]
            la_end_token = annotation["long_answer"]["end_token"] - 1
            while orig_to_tok_index[la_start_token] == -1:
                la_start_token += 1
            while orig_to_tok_end_index[la_end_token] == -1:
                la_end_token -= 1

            assert la_start_token <= la_end_token

            la_tok_start_position = orig_to_tok_index[la_start_token]
            la_tok_end_position = orig_to_tok_end_index[la_end_token] - 1

            if annotation["yes_no_answer"].lower() == "none":
                example_answer_type = AnswerType.LONG
            elif annotation["yes_no_answer"].lower() == "yes":
                example_answer_type = AnswerType.YES
            elif annotation["yes_no_answer"].lower() == "no":
                example_answer_type = AnswerType.NO
            else:
                assert False

            if "short_answers" in annotation and len(annotation["short_answers"]) > 0:
                sa_tok_start_position = len(all_doc_tokens) + 1
                sa_tok_end_position = -1
                example_answer_type = AnswerType.SHORT
                for short_answer in annotation["short_answers"]:
                    sa_start_token = short_answer["start_token"]
                    sa_end_token = short_answer["end_token"] - 1

                    sa_tok_start_position = min(sa_tok_start_position, orig_to_tok_index[sa_start_token])
                    sa_tok_end_position = max(sa_tok_end_position, orig_to_tok_end_index[sa_end_token] - 1)
                    assert sa_tok_start_position >= 0 and sa_tok_end_position >= 0
                assert la_tok_start_position <= sa_tok_start_position and la_tok_end_position >= sa_tok_end_position
                la_tok_start_position = sa_tok_start_position
                la_tok_end_position = sa_tok_end_position
            else:
                sa_tok_start_position = -1
                sa_tok_end_position = -1
        else:
            example_answer_type = AnswerType.UNKNOWN
            la_tok_start_position = -1
            la_tok_end_position = -1
            sa_tok_start_position = -1
            sa_tok_end_position = -1

    # The -3 accounts for [CLS], [SEP] and [SEP]
    max_tokens_for_doc = max_seq_length - len(query_tokens) - 3

    # We can have documents that are longer than the maximum sequence length.
    # To deal with this we do a sliding window approach, where we take chunks
    # of the up to our max length with a stride of `doc_stride`.
    _DocSpan = collections.namedtuple(  # pylint: disable=invalid-name
        "DocSpan", ["start", "length"])
    doc_spans = []
    start_offset = 0
    logger.info("ALL DOC_TOKENS %s" %" ".join([str(x) for x in all_doc_tokens][:50]))
    while start_offset < len(all_doc_tokens):
        length = len(all_doc_tokens) - start_offset
        if length > max_tokens_for_doc:
            length = max_tokens_for_doc
        doc_spans.append(_DocSpan(start=start_offset, length=length))
        if start_offset + length == len(all_doc_tokens):
            break
        start_offset += min(length, doc_stride)

    for (doc_span_index, doc_span) in enumerate(doc_spans):
        tokens = []
        token_to_orig_map = {}
        token_is_max_context = {}
        segment_ids = []
        tokens.append("[CLS]")
        segment_ids.append(0)
        for token in query_tokens:
            tokens.append(token)
            segment_ids.append(0)
        tokens.append("[SEP]")
        segment_ids.append(0)

        is_sentence_end = [True] + [False] * (len(tokens) - 3) + [True] + [False]
        is_paragraph_end = [True] + [False] * (len(tokens) - 3) + [True] + [False]
        orig_tok_idx = [-1] * len(tokens)
        candidate_idx = [-1] * len(tokens)

        for i in range(doc_span.length):
            split_token_index = doc_span.start + i
            token_to_orig_map[len(tokens)] = tok_to_orig_index[split_token_index]

            is_max_context = _check_is_max_context(doc_spans, doc_span_index,
                                                    split_token_index)
            token_is_max_context[len(tokens)] = is_max_context
            tokens.append(all_doc_tokens[split_token_index])
            segment_ids.append(1)

            is_sentence_end.append(tok_is_sentence_end[split_token_index])
            is_paragraph_end.append(tok_is_paragraph_end[split_token_index])
            orig_tok_idx.append(tok_to_orig_index[split_token_index])
            candidate_idx.append(tok_candidate_idx[split_token_index])

        is_sentence_end[-1] = False
        is_paragraph_end[-1] = False

        tokens.append("[SEP]")
        segment_ids.append(1)

        is_sentence_end.append(True)
        is_paragraph_end.append(True)
        orig_tok_idx.append(-1)
        candidate_idx.append(-1)

        assert len(orig_tok_idx) == len(tokens)
        assert len(is_sentence_end) == len(tokens)
        assert len(is_paragraph_end) == len(tokens)
        assert len(candidate_idx) == len(tokens)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        la_start_position = None
        la_end_position = None
        sa_start_position = None
        sa_end_position = None
        answer_type = None
        if is_training:
            # For training, if our document chunk does not contain an annotation
            # we throw it out, since there is nothing to predict.
            doc_start = doc_span.start
            doc_end = doc_span.start + doc_span.length - 1
            out_of_span = False
            if not (la_tok_start_position >= doc_start and
                    la_tok_end_position <= doc_end):
                out_of_span = True

            if out_of_span:
                if random.random() > include_unknowns:
                    continue
                answer_type = AnswerType.UNKNOWN
                la_start_position = 0
                la_end_position = 0
                sa_start_position = 0
                sa_end_position = 0
            else:
                answer_type = example_answer_type
                doc_offset = len(query_tokens) + 2
                la_start_position = la_tok_start_position - doc_start + doc_offset
                la_end_position = la_tok_end_position - doc_start + doc_offset

                if not (sa_tok_start_position >= doc_start and sa_tok_end_position <= doc_end):
                    sa_start_position = -1
                    sa_end_position = -1
                else:
                    sa_start_position = sa_tok_start_position - doc_start + doc_offset
                    sa_end_position = sa_tok_end_position - doc_start + doc_offset

            assert la_start_position <= la_end_position
            assert sa_start_position <= sa_end_position

        if example_index >= 0:
            logger.info("*** Example ***")
            logger.info("unique_id: %s" % (unique_id))
            logger.info("example_index: %s" % (example_index))
            logger.info("doc_span_index: %s" % (doc_span_index))
            print("token list:",tokens[:20])
            # print("token list size", len(tokens))
            logger.info("tokens: %s" % " ".join(tokens)[:20])
            logger.info("token_to_orig_map: %s" % " ".join([
                "%d:%d" % (x, y) for (x, y) in token_to_orig_map.items()][:20]))
            logger.info("token_is_max_context: %s" % " ".join([
                "%d:%s" % (x, y) for (x, y) in token_is_max_context.items()
            ][:20]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids][:20]))
            logger.info(
                "input_mask: %s" % " ".join([str(x) for x in input_mask][:20]))
            logger.info(
                "segment_ids: %s" % " ".join([str(x) for x in segment_ids][:20]))
            if is_training and answer_type != AnswerType.UNKNOWN:
                answer_text = " ".join(tokens[la_start_position:(la_end_position + 1)][:20])
                logger.info("la_start_position: %d" % (la_start_position))
                logger.info("la_end_position: %d" % (la_end_position))
                logger.info(
                    "answer: %s" % (answer_text))
        doc_tree = get_doc_tree(is_sentence_end, is_paragraph_end, orig_tok_idx, candidate_idx)

        graph, start_positions, end_positions = build_graph(doc_tree, len(is_sentence_end),
                                                            la_start_position, la_end_position,
                                                            sa_start_position, sa_end_position)

        features.append(
            InputFeatures(
                unique_id=unique_id,
                example_id=example.example_id,
                doc_span_index=doc_span_index,
                tokens=None,
                token_to_orig_map=token_to_orig_map,
                token_is_max_context=token_is_max_context,
                input_ids=input_ids,
                input_mask=input_mask,
                segment_ids=segment_ids,
                graph=graph,
                answer_type=answer_type,
                start_positions=start_positions,
                end_positions=end_positions))
        unique_id += 1


12/30/2020 15:35:53 - INFO - src_nq.create_examples -   ALL DOC_TOKENS [unused0] per ##se ##phone goddess of the underworld , spring ##time , flowers and vegetation statue of per ##se ##phone with a sis ##trum . her ##ak ##lion archaeological museum , crete ab ##ode the underworld , sicily , mount olympus symbol po ##me ##gra ##nate , seeds of grain
12/30/2020 15:35:53 - INFO - src_nq.create_examples -   *** Example ***
12/30/2020 15:35:53 - INFO - src_nq.create_examples -   unique_id: 1000000000
12/30/2020 15:35:53 - INFO - src_nq.create_examples -   example_index: 0
12/30/2020 15:35:53 - INFO - src_nq.create_examples -   doc_span_index: 2
12/30/2020 15:35:53 - INFO - src_nq.create_examples -   tokens: [CLS] in greek mytho
12/30/2020 15:35:53 - INFO - src_nq.create_examples -   token_to_orig_map: 12:303 13:303 14:306 15:306 16:306 17:309 18:309 19:309 20:309 21:309 22:316 23:316 24:317 25:318 26:319 27:323 28:323 29:330 30:330 31:330
12/30/2020 15:35:53 - INFO - src_nq.create_examples

In [63]:
segment_ids
tokens
input_ids
doc_tree
is_sentence_end
is_paragraph_end
example.annotation
candidate_idx
orig_tok_idx
doc_tree



[(-1, [(-1, [(-1, 0)])]),
 (-1,
  [(-1,
    [(-1, 1),
     (-1, 2),
     (-1, 3),
     (-1, 4),
     (-1, 5),
     (-1, 6),
     (-1, 7),
     (-1, 8),
     (-1, 9),
     (-1, 10)])]),
 (62,
  [(-1,
    [(-1, 11),
     (1502, 12),
     (1503, 13),
     (1503, 14),
     (1503, 15),
     (1504, 16),
     (1505, 17)])]),
 (63,
  [(-1,
    [(-1, 18),
     (1508, 19),
     (1508, 20),
     (1508, 21),
     (1508, 22),
     (1509, 23),
     (1510, 24),
     (1510, 25),
     (1510, 26),
     (1510, 27),
     (1510, 28),
     (1510, 29),
     (1510, 30),
     (1510, 31),
     (1510, 32),
     (1510, 33),
     (1511, 34),
     (1512, 35),
     (1513, 36),
     (1514, 37),
     (1515, 38),
     (1516, 39),
     (1516, 40),
     (1517, 41),
     (1518, 42),
     (1518, 43),
     (1518, 44),
     (1519, 45),
     (1520, 46),
     (1521, 47),
     (1521, 48),
     (1522, 49)]),
   (-1,
    [(1523, 50),
     (1524, 51),
     (1525, 52),
     (1526, 53),
     (1527, 54),
     (1528, 55),
     (1529, 

In [321]:
doc_spans
example = part[0]
example.doc_tokens[0:20]
# [x for x in example.doc_tokens if x['token']=='episode']
[x for x in example.doc_tokens if x['start_byte']==3113]



[]

In [116]:
for i in range(10):
    print(part[i].question_tokens)

['when', 'is', 'the', 'last', 'episode', 'of', 'season', '8', 'of', 'the', 'walking', 'dead']
['when', 'is', 'the', 'last', 'episode', 'of', 'season', '8', 'of', 'the', 'walking', 'dead']
['in', 'greek', 'mythology', 'who', 'was', 'the', 'goddess', 'of', 'spring', 'growth']
['when', 'is', 'the', 'last', 'episode', 'of', 'season', '8', 'of', 'the', 'walking', 'dead']
['benefits', 'of', 'colonial', 'life', 'for', 'single', 'celled', 'organisms']
['when', 'is', 'the', 'last', 'episode', 'of', 'season', '8', 'of', 'the', 'walking', 'dead']
['how', 'many', 'season', 'of', 'the', 'man', 'in', 'the', 'high', 'castle']
['when', 'is', 'the', 'last', 'episode', 'of', 'season', '8', 'of', 'the', 'walking', 'dead']
['who', 'was', 'the', 'first', 'ministry', 'head', 'of', 'state', 'in', 'nigeria']
['when', 'is', 'the', 'last', 'episode', 'of', 'season', '8', 'of', 'the', 'walking', 'dead']


In [121]:
features[1].unique_id

1000000001

In [123]:
features[1].example_id

4549465242785278785

In [124]:
features[1].doc_span_index

22

In [132]:
features[1].tokens

In [148]:
# features[1].token_to_orig_map
# features[1].token_is_max_context
# features[1].segment_ids
# features[1].answer_type
features[1].start_positions
features[1].end_positions

[397, 13, 2]

In [153]:
tokenizer.tokenize("hello yoho sb")

['hello', 'yo', '##ho', 'sb']

In [181]:
example = examples[0]
sorted_la_candidates = sorted(enumerate(example.la_candidates), key=lambda x: x[1]["start_token"])
sorted_la_candidates[0]


(0,
 {'end_byte': 57620,
  'end_token': 216,
  'start_byte': 53609,
  'start_token': 24,
  'top_level': True})

In [183]:
candidate = sorted_la_candidates[0][1]
start_index = candidate["start_token"]
end_index = candidate["end_token"]

context_tokens = []
context_orig_idx = []

for tok_idx in range(start_index, end_index):
    token_item = example.doc_tokens[tok_idx]
    if not token_item["html_token"]:
        context_tokens.append(token_item["token"].replace(" ", ""))
        if context_tokens[-1] == "":
            context_tokens = context_tokens[:-1]
            continue
        context_orig_idx.append(tok_idx)

In [191]:
example.doc_tokens[31]

{'end_byte': 53868,
 'html_token': False,
 'start_byte': 53862,
 'token': 'season'}

In [189]:
context_tokens[:20]

['The',
 'Walking',
 'Dead',
 '(',
 'season',
 '8',
 ')',
 'Promotional',
 'poster',
 'Starring',
 'Andrew',
 'Lincoln',
 'Norman',
 'Reedus',
 'Lauren',
 'Cohan',
 'Chandler',
 'Riggs',
 'Danai',
 'Gurira']

In [188]:
context_orig_idx[:20]


[27,
 28,
 29,
 30,
 31,
 32,
 33,
 38,
 39,
 44,
 49,
 50,
 53,
 54,
 57,
 58,
 61,
 62,
 65,
 66]

In [198]:
counts = {"Table": 0, "Paragraph": 0, "List": 0, "Other": 0}

sorted_la_candidates = sorted(enumerate(example.la_candidates), key=lambda x: x[1]["start_token"])

tok_is_sentence_end = []
tok_is_paragraph_end = []
tok_candidate_idx = []
tok_to_orig_index = []
orig_to_tok_index = [-1] * len(example.doc_tokens)
orig_to_tok_end_index = [-1] * len(example.doc_tokens)
all_doc_tokens = []

for candidate_idx, candidate in sorted_la_candidates:
    if not candidate["top_level"]:
        continue
    start_index = candidate["start_token"]
    end_index = candidate["end_token"]

    context_tokens = []
    context_orig_idx = []

    for tok_idx in range(start_index, end_index):
        token_item = example.doc_tokens[tok_idx]
        if not token_item["html_token"]:
            context_tokens.append(token_item["token"].replace(" ", ""))
            if context_tokens[-1] == "":
                context_tokens = context_tokens[:-1]
                continue
            context_orig_idx.append(tok_idx)

    if len(context_tokens) == 0:
        continue

    all_doc_tokens.append(
        "[unused{}]".format(
            get_candidate_type(example.doc_tokens[start_index]["token"], counts, max_position)))
    tok_to_orig_index.append(-1)
    tok_is_sentence_end.append(False)
    tok_is_paragraph_end.append(False)
    tok_candidate_idx.append(candidate_idx)

In [244]:
xs = [ca for ca in sorted_la_candidates if ca[1]["top_level"]]
len(xs)
all_doc_tokens
tok_is_sentence_end
tok_is_paragraph_end
tok_candidate_idx
context_tokens
context = " ".join(context_tokens)
context
context_sentences = nlp(context).sents
context_idx = 0
orig_to_tok_index[context_orig_idx[context_idx]] = len(all_doc_tokens)
for sentence in context_sentences:
    sentence_len = 0
    for token in sentence.string.strip().split():
        # print(token)
        pass

context_tokens

['^',
 '1',
 'Live',
 '+',
 '7',
 'ratings',
 'were',
 'not',
 'available',
 ',',
 'so',
 'Live',
 '+',
 '3',
 'ratings',
 'have',
 'been',
 'used',
 'instead',
 '.']

In [262]:
[(features[i].example_id, features[i].doc_span_index) for i in range(20)]

[(4549465242785278785, 21),
 (4549465242785278785, 22),
 (4549465242785278785, 23),
 (4549465242785278785, 24),
 (4549465242785278785, 22),
 (4549465242785278785, 23),
 (4549465242785278785, 24),
 (-2543388002166163252, 2),
 (-2543388002166163252, 3),
 (-2543388002166163252, 4),
 (-2543388002166163252, 8),
 (-2543388002166163252, 35),
 (-2543388002166163252, 63),
 (4549465242785278785, 22),
 (4549465242785278785, 23),
 (4549465242785278785, 24),
 (4549465242785278785, 5),
 (4549465242785278785, 22),
 (4549465242785278785, 23),
 (4549465242785278785, 24)]

In [263]:
doc_spans

[DocSpan(start=0, length=497),
 DocSpan(start=128, length=497),
 DocSpan(start=256, length=497),
 DocSpan(start=384, length=497),
 DocSpan(start=512, length=497),
 DocSpan(start=640, length=497),
 DocSpan(start=768, length=497),
 DocSpan(start=896, length=497),
 DocSpan(start=1024, length=497),
 DocSpan(start=1152, length=497),
 DocSpan(start=1280, length=497),
 DocSpan(start=1408, length=497),
 DocSpan(start=1536, length=497),
 DocSpan(start=1664, length=497),
 DocSpan(start=1792, length=497),
 DocSpan(start=1920, length=497),
 DocSpan(start=2048, length=497),
 DocSpan(start=2176, length=497),
 DocSpan(start=2304, length=497),
 DocSpan(start=2432, length=497),
 DocSpan(start=2560, length=497),
 DocSpan(start=2688, length=497),
 DocSpan(start=2816, length=497),
 DocSpan(start=2944, length=497),
 DocSpan(start=3072, length=497),
 DocSpan(start=3200, length=497),
 DocSpan(start=3328, length=461)]